# Robotics Foundations (H) - Lab 1

## ILOs
In this lab, you will learn and understand the streaming data transport mechanisms of ROS, ROS package creation, file system and compilation tools, available robot sensors and actuators, and the simulation framework of Gazebo. At the end of the lab, you should be able to:

* Use the simulation facilities in ROS
* Create a ROS package, write your first ROS node and publish a data stream on a ROS topic.
* Develop an open-loop control system to start and stop a simulated robot based on a simple timer
* Read information from a data stream of a ROS topic and employ ROS graphical and text-based utilities for inspecting ROS topics
* Develop a closed-loop control between sensing and actuation by writing a ROS node that enables a robot to avoid obstacles and randomly wander around the environment

** NOTE: Remember to save/commit your work of this lab in GitHub or any other cloud service. As soon as you exit the Docker container, all your files and edits will be lost!**

## ROS in a nutshell!
The main aim of ROS is **to provide operating system-like functionalities for specialised robotic agents, under an open-source license**. The creators based ROS around five fundamental design principles: peer-to-peer, tools-based, multi-lingual, thin, and open-source. It currently supports four languages: C++, Python, Octave and LISP with other language ports in development. To be language-independent, ROS uses a language-neutral interface definition language to define messages sent between modules. So, the services it offers are similar to the ones found in most software middleware (like Django in WAD2!):

- Hardware abstraction, which enables the development of portable code which can interface with a large number of robot platforms
- Low-level device control, facilitating robotic control
- Inter-process communication via message passing
- Software package management, which ensures the framework is easily extensible

Thus, at the core of all the programs running on ROS is an anonymised form of the publisher-subscriber pattern. Processes are termed **nodes** in ROS, as they form part of an underlying graph (Figure 1) that keeps track of nodes (via the graph resource name, a unique identifier) and all the communications between them at a fine-grained scale. Since most robotic systems are assumed to comprise numerous components, all requiring control or I/O services, the ROS node architecture expects that each computation is delegated to a separate node, thus reducing the complexity involved in developing monolithic control systems. 

![ros_graph.png](imgs/ros_graph.png)
<div style="text-align:center"><b>Figure 1: Example of a ROS node graph</b><br></div>

A benefit of using ROS is that it allows debugging of a node in active development alongside working nodes, due to the modular structure. It also means people working in robotics do not have to reinvent the wheel for each project and software developed for robots can be easily shared creating a vibrant community. Additionally, all other nodes can be started and left running, and developers can just edit and restart the one node they are working on. As the system gets more complex this can be incredibly beneficial ([http://www.willowgarage.com/sites/default/files/icraoss09-ROS.pdf](http://www.willowgarage.com/sites/default/files/icraoss09-ROS.pdf)).

Newly-created nodes can communicate in one of three ways:

- Topics
- RPC Services
- Parameter Server

The most common communication mechanism involves transmitting and receiving messages asynchronously on a strongly-typed named bus or log called a **topic**. Nodes can create, publish or subscribe to a topic, and send and receive data in the form of messages of a predefined type, at a set frequency measured in Hz. Built for unidirectional streaming communication topics also benefit from anonymous publish/subscribe semantics, thus removing communicating nodes’ awareness of each other.

However, in cases where the many-to-many transport provided by topics is not appropriate, ROS offers a form of node interaction based on the request/reply paradigm of remote procedure calls. These **services** are defined by a pair of messages, of which one is the request sent to a node, and the other is the reply.

The **parameter server** takes the form of a globally-viewable dictionary of typically static, non-binary data points. The server is designed primarily for the storage of shared configuration parameters, and does not uphold any premise of high performance and thus should not be seen as a global variable store.

The above 3 communication types support the smooth integration of custom user code within a ROS ecosystem and unify the different APIs that would typically be needed to access relevant system information (sensor data, actuator positions, etc.).

## ROS Filesystem
Similar to an operating system, a ROS workspace is divided into folders, and these folders have files that describe their functionalities:

* **Packages**: Packages form the atomic level of ROS. A package has the minimum structure and content to create a program within ROS. It may have ROS runtime processes (nodes), configuration files, and so on. In the following sections, you will create a ROS package.
* **Package manifests**: Package manifests provide information about a package, licenses, dependencies, compilation flags, and so on. A package manifest is managed with a file called package.xml.
* **Metapackages**: When you want to aggregate several packages in a group, you will use meta-packages. In ROS, there exists a lot of these meta-packages; for example, the navigation software stack.
* **Metapackage manifests**: Metapackage manifests ( package.xml ) are similar to a normal package, but with an export tag in XML. It also has certain restrictions in its structure.
* **Message (msg) types**: A message is the information that a process sends to other processes. ROS has a lot of standard types of messages. Message descriptions are stored in my_package/msg/MyMessageType.msg.
* **Service (srv) types**: Service descriptions, stored in my_package/srv/MyServiceType.srv, define the request and response data structures for services provided by each process in ROS.

## Creating a ROS package
To develop ROS robots, you usually create a workspace where you will store ROS packages and related configurations and files. In general terms, the workspace is a folder tree that contains packages, those packages include your source files, and the environment or workspace provides us with a way to compile those packages. It is useful when you want to compile various packages at the same time, and it is a good way of centralising all of your developments.

So, to create your workspace directory, which you will be located in: `~/robot_ws` (`~` is a shortcut to your user's home directory), type the following in a terminal (see [Note 1](#note1) on how to open a terminal and [Note 2](#note2)):

``` bash
mkdir -p robot_ws/src
cd robot_ws/src/
catkin_init_workspace
```

With the above commands, you have also created an `src` folder where your ROS package and python scripts will be stored. Finally, you need to run one more command to create a package in the new workspace. To create a package called `myturtlebot` that uses `rospy` (the Python client for ROS) and a few standard ROS message packages, you will use the `catkin_create_pkg` command:

``` bash
catkin_create_pkg myturtlebot rospy geometry_msgs sensor_msgs
```
The first argument, `myturtlebot`, is the name of your new package. The following arguments are the names of packages that the new package depends on. You must include these because the ROS build system needs to know the package dependencies to keep the builds up-to-date efficiently and to generate any required dependencies. **Tip:** you can find out more about these ROS message packages and other packages by searching for them in Google, e.g. https://www.google.co.uk/search?q=geometry_msgs+ros

After running the `catkin_create_pkg command`, there will be a package directory called `myturtlebot` inside the workspace (see Figure below), and includes the following files:

* `~/robot_ws/src/myturtlebot/CMakeLists.txt`, a starting point for the build script for this package
* `package.xml`, a machine-readable description of the package, including details such as its name, description, author, license, and which other packages it depends on to build and run (as described in previous sections)

![filestructure.png](imgs/filestructure.png)
<div style="text-align:center"><b>Figure 2: ROS package structure</b><br></div>

## Creating a ROS node
You can now create a minimal ROS node inside of the `myturtlebot` package. The purpose of this node will be to communicate with a simulated robot (more in the following sections) to move it around an environment. The following code will send a stream of motion commands 10 times per second, alternating every 3 seconds between driving and stopping. When driving, the program will send forward velocity commands of 0.5 meters per second. When stopped, it will send commands of 0 meters per second. First, please open the following Python ROS node and examine the code.

<a href="http://localhost:8888/edit/notebooks/move.py" target="_blank">Open move.py</a>

A lot of this code is just setting up the system and its data structures. The most important function of this program is to change behaviour every 3 seconds from driving to stopping. This is performed by the three-line block below, which uses [rospy.Time](http://wiki.ros.org/rospy/Overview/Time) to measure the duration since the last change of behaviour:

``` python
if light_change_time > rospy.Time.now():
    driving_forward = not driving_forward
    light_change_time = rospy.Time.now() + rospy.Duration(3)
```
    
Now, create a Python script inside 'myturtlebot/src' and copy and paste the above code. Alternatively, press Ctrl+Enter or Shift+Enter to evaluate the following cell:

In [1]:
%%bash
cp ~/notebooks/move.py ~/robot_ws/src/myturtlebot/src/move.py

Like all Python scripts, it is convenient to make it executable so ROS can run the node (select the following cell and press):

In [2]:
%%bash
chmod +x ~/robot_ws/src/myturtlebot/src/move.py

Now, you can use your program to control a simulated robot. But first, you need to install the Turtlebot simulation stack -- this operation might take a while... time for a break? : ) :

``` bash
sudo apt install ros-kinetic-turtlebot-gazebo
```

you now need to "compile" your package to register it within the ROS `PATH`:

``` bash
cd ~/robot_ws
catkin_make
source devel/setup.bash
```

The last line adds all the packages within `robot_ws` to the ROS PATH, so ROS knows where to look for packages and nodes! Now, you are ready to instantiate a Turtlebot in the simulator. You’ll use a simple simulated world to start with, by typing this in a new terminal window (remember to hit the Tab key often when typing ROS shell commands for autocompletion) (see [Note 3](#note3) if you do not want to open multiple terminals from Jupyter):

``` bash
roslaunch turtlebot_gazebo turtlebot_world.launch
```

[`roslaunch`](http://wiki.ros.org/roslaunch) is a tool that allows you to run several ROS nodes, declare ROS parameters, pass command line arguments to ROS nodes, and so on. A `roslaunch` file is structured as [XML](http://wiki.ros.org/roslaunch/XML) and run a [`roscore`](http://wiki.ros.org/roscore) in the background. The `roscore` is the central hub where topics, services and parameters are stored and must be running so ROS nodes can communicate between each other.

After the simulation finished loading, it is time to fire up your control node (if `rosrun` cannot find your `myturtlebot` package, run `source devel/setup.bash` while you are in your workspace, i.e. `~/robot_ws`):

``` bash
rosrun myturtlebot move.py cmd_vel:=cmd_vel_mux/input/teleop
```

The `cmd_vel` argument is remapping ([more about remapping here](http://wiki.ros.org/Remapping%20Arguments)) is your Twist topic to the topic your ROS node is expecting (see Line 10 in `move.py`). Although you could have declared `cmd_vel_pub` to publish to this topic in the `move.py` code, your goal is to write ROS nodes that are as generic as possible, and in this case, you can easily remap `cmd_vel` to whatever is required by any ROS package.

When `move.py` is running, you should now see a Turtlebot alternating every second between driving forward and stopping. When you get bored with it, just give a Ctrl-C to the newly created node as well as the TurtleBot simulation to stop your node and the simulation, respectively.

## Sensing
Now, let's find out how to stream sensor data into ROS nodes. For this, you will use sensor data from an onboard laser scanner in the (simulated) Turtlebot. Whenever you want to receive a topic in ROS, it’s often helpful to first just echo it to the console, to make sure that it is being published under the topic name you expect and to confirm that you understand the data type.

In the case of Turtlebot, you want a laser scan: a linear vector of ranges from the robot to the nearest obstacles in various directions. To save on cost, sadly, the Turtlebot does not have a real laser scanner. It does, however, have a Kinect depth camera, and the Turtlebot software stack extracts the middle rows of the Kinect’s depth image, does a bit of filtering, and then publishes the data as [`sensor_msgs/LaserScan`](http://docs.ros.org/api/sensor_msgs/html/msg/LaserScan.html) messages on the `/scan` topic. This means that from the standpoint of the high-level software, the data shows up exactly like “real” laser scans on more expensive robots. Although the Turtlebot can perceive the obstacle directly in front of it, the obstacle on its right side is mostly out of view. Such are the trade-offs involved with using low-cost depth cameras as navigation sensors!

To start using the sensor data, you can just dump the scan topic to the console to verify that the simulated laser scanner is working. First, fire up a Turtlebot simulation, if one isn’t already running:

``` bash
roslaunch turtlebot_gazebo turtlebot_world.launch
```

You will now query all the topics that are currently being advertised (press Shift+Enter to evaluate the cell):

In [ ]:
%%bash
rostopic list

Towards the end of the list, you should be able to see the `/scan` topic. Now let's see what that topic contains:

In [ ]:
%%bash
rostopic echo scan -n 3

This will print a continuous stream of text representing the LaserScan messages (for practical reasons, only 3 messages are displayed here!). If you run the above command in a terminal, press Ctrl-C to stop it. To find more about `rostopic`, [go here!](http://wiki.ros.org/rostopic). You will find out that `rostopic` is a very neat tool for inspecting and getting message types of available topics within a ROS system. 

Most of the text is the LaserScan message being published into the `/scan` topic, which is what you are interested in: the ranges array contains the range from the Turtlebot to the nearest object at bearings easily computed from the ranges array index. Specifically, if the message instance is named `msg`, you can compute the bearing for a particular range estimate as follows, where `i` is the index into the ranges array:

``` python
bearing = msg.angle_min + i * msg.angle_max / len(msg.ranges)
```

To retrieve the range to the nearest obstacle directly in front of the robot, you will select the middle element of the ranges array:

``` python
range_ahead = msg.ranges[len(msg.ranges)/2]
```

Or, to return the range of the closest obstacle detected by the scanner:

``` python
closest_range = min(msg.ranges)
```

For instance, this is how it would look like in a ROS python node.

``` python
# Import ROS Python API
import rospy

# Data structure here: http://docs.ros.org/api/sensor_msgs/html/msg/LaserScan.html
from sensor_msgs.msg import LaserScan

# This function is called each time a new message arrives on the scan topic. This callback function then prints the range measured to the object directly in front of the robot by picking the middle element of the ranges field of the LaserScan message
def scan_callback(msg):
    bearing = msg.angle_min + i * msg.angle_max / len(msg.ranges)
    print "bearing: %0.1f" % bearing
    
    range_ahead = msg.ranges[len(msg.ranges)/2]
    print "range ahead: %0.1f" % range_ahead
    
    closest_range = min(msg.ranges)
    print "closest range: %0.1f" % closest_range

rospy.init_node('reading_ranges', anonymous=True)

# You now subscribe to the topic, this is similar to 'rostopic echo /scan' but now the messages are chaneled to 'scan_callback' rather than the screen
scan_sub = rospy.Subscriber('scan', LaserScan, scan_callback)

rate = rospy.Rate(1) # In Hz, where Hz are 1/seconds.

for i in range(0,1):
    rate.sleep() # Sleep for 1 second, but will received all streamed messages from /scan within this time

scan_sub.unregister()
```

This signal chain is deceptively complex: you are picking out elements of an emulated laser scan, which is itself produced by picking out a few of the middle rows of the Turtlebot’s Kinect depth camera, which is itself generated in Gazebo by back-projecting rays into a simulated environment! It’s hard to overemphasise the utility of simulation for robot software development.

Alternatively, you can run a ROS node that reads Laser scan ranges. To do this, copy and paste the following code into a ROS python node within your `myturtlebot`'s `src` package.

``` python
#!/usr/bin/env python
import rospy

# Data structure here: http://docs.ros.org/api/sensor_msgs/html/msg/LaserScan.html
from sensor_msgs.msg import LaserScan

# This function is called each time a new message arrives on the scan topic. This callback function then prints the range measured to the object directly in front of the robot by picking the middle element of the ranges field of the LaserScan message
def scan_callback(msg):
    range_ahead = msg.ranges[len(msg.ranges)/2]
    print "range ahead: %0.1f" % range_ahead

rospy.init_node('reading_ranges')

# you now subscribe to the topic, this is similar to 'rostopic echo /scan' but now the messages are channelled to 'scan_callback' rather than the screen
scan_sub = rospy.Subscriber('scan', LaserScan, scan_callback)

# spin() keeps your node from exiting until the node is shut down. This is thread independent and does not affect the execution of the callbacks
rospy.spin()
```

This time, you will be using a GUI for visualising the state of robots -- this is like looking into the "brains" of the robot since you can inspect and query topics, services and what the robot perceives! To do this, run the following commands, each in a different console:

``` bash
catkin_make
roslaunch turtlebot_gazebo turtlebot_world.launch
rosrun rviz rviz
rosrun turtlebot reading_ranges.py
```

Remember to `chmod` your newly created python script! After the simulation and [RViz](http://wiki.ros.org/rviz) have started, focus on RViz and select `base_link` in the `Global Options->Fixed Frame list`. Then click `Add` and select the `By topic` tab, scroll down until you find the `/scan` topic. Finally, click `Add` again select `RobotModel` -- you should get something similar as shown in Figure 3. If you click `Add` again and select the `By topic` tab, you can explore all available topics that you can display within RVIZ, why don't you subscribe to the `PointCloud` topic and see what happens!

![rviz.png](imgs/rviz.png)
<div style="text-align:center"><b>Figure 3: Turtlebot visualisation</b><br></div>

You can experiment with `reading_ranges.py` in Gazebo by dragging and rotating the Turtlebot around in the world. Click the Move icon (see Figure 4) in the Gazebo toolbar to enter the Move mode, and then click and drag the Turtlebot around the scene. The terminal running `reading_ranges.py` will print a changing stream of numbers indicating the range (in meters) from the Turtlebot to the nearest obstacle (if any) directly in front of it. Gazebo also has a Rotate tool that will (by default) rotate a model about its vertical axis. Both the Move and Rotate tools will immediately affect the output of the `reading_ranges.py` node, since the simulation (by default) stays running while models are being dragged and rotated.

![zoommove.png](imgs/zoommove.png)
<div style="text-align:center"><b>Figure 4: Gazebo tools for moving and rotating objects within the simulation.</b><br></div>

## Sensing and Actuation (Exercise)

Let's now add a bit of intelligence to your `myturtlebot`. From `move.py` which implements an open-loop control to drive the robot, and `reading_ranges.py` which processes a sensor stream to estimate the range to the nearest object directly in front of the turtlebot, your job is to put together both python scripts to let the robot to go straight until it sees an obstacle within 0.8 metres or time out after 5 seconds. In either case, the Turtlebot should stop and rotate 1 radian (approx. 52.7 degrees) to change its direction. To rotate Turtlebot, you will use the angular component of the [Twist message](http://docs.ros.org/api/geometry_msgs/html/msg/Twist.html), and you should rotate the robot on the Z axis (e.g. `twist.angular.z`). Angular values are given in radians! Your node should be named `wander.py`.

**Tip:** Instead of defining a functioning for the callback of the subscriber, you may want to declare a class to follow good software engineering practices. This class might have the following structure:

``` python
class ReadScan:
    
    range_value = 1 # anything to start with

    def query_range(self):
        return self.range_value

    def scan_callback(self, msg):
        import math
        self.range_value = min(msg.ranges)
        # If the object is not within the sensor's working range, range_value will be NaN; so let's set it to 0
        if math.isnan(self.range_value):
            self.range_value = 0
        rospy.loginfo(self.range_value)
```

To subscribe to the `/scan` topic and publish the Twist message, you may use the following:

``` python
scan_sub = rospy.Subscriber('scan', LaserScan, "<<replace here with your callback>>")
cmd_vel_pub = rospy.Publisher('cmd_vel', Twist, queue_size=1)
```

Finally, when the Turtlebot is rotating, you should set driving forward to zero, e.g.

``` python
twist.linear.x = 0
twist.angular.z = 1
```

After you have finished writing your `wander.py` ROS node, do:

``` bash
chmod +x ~/robot_ws/src/myturtlebot/src/wander.py
rosrun myturtlebot wander.py cmd_vel:=cmd_vel_mux/input/teleop
```

Turtlebot will keep doing these two things until you stop the script with Ctrl-C. If you get bored, you can change the parameters, play around with the script, so it does something more intelligent (e.g. maintain a safe distance from the obstacle while surrounding it).

## Working with Services
Services, as explained in previous sections, are RPC calls within the ROS ecosystem. For the purpose of this lab, let's create a service that queries the current range reading and the status of the robot (whether it is driving forward or turning) and allows to change the status of the robot on-the-fly. The first thing is that you need to create a new directory within your `myturtlebot` package called `srv`, so open a console and type (alternatively, you can use jupyter to create the folder and the service file):

``` bash
cd ~/robot_ws/src/myturtlebot
mkdir srv
cd srv
touch TurtlebotStatus.srv
```

Now, open `TurtlebotStatus.srv` with a text editor (i.e. `vi`, `nano` or in jupyter) and add the following:

``` yaml
bool forwardrqst
---
float32 range
bool forwardresp
```

[ROS Services](http://wiki.ros.org/Services) are similar to ROS topics and messages; the main difference is the `---` separation between declarations. The part above is the *request* whereas the part below is the *response* - messages have the part above only. After defining your service, you need to tell ROS that it needs to compile it and do the required plumbing so you can access it within your python script. To do this, open `package.xml` in your `myturtlebot` package and add the next two lines in it:

``` xml
<build_depend>message_generation</build_depend>
<exec_depend>message_runtime</exec_depend>
```

then, you need to add the `message_generation` dependency to generate messages in `CMakeLists.txt`, so open this file and add `message_generation` as follows:

```
find_package(catkin REQUIRED COMPONENTS
  geometry_msgs
  rospy
  sensor_msgs
  message_generation
)
```

Next, find the `add_service_files` CMake directive in `CMakeLists.txt`, uncomment/replace with the following:

```
add_service_files(
    FILES
    TurtlebotStatus.srv
)
```

and, the same for:

```
generate_messages(
   DEPENDENCIES
   geometry_msgs
   sensor_msgs
   std_msgs
)
```

`generate_messages` will tell `catkin_make` to generate code for use in all supported languages in ROS, in your case, you will use the Python variant. Finally, let's compile and add it to ROS path! So in a console, type:

``` bash
cd ~/robot_ws
catkin_make && source devel/setup.bash
```

To verify that everything is OK, you can use the `rossrv` command line tool to show your service (press Shit+Enter in the following cells):

In [ ]:
%%bash
source ~/robot_ws/devel/setup.bash
rossrv show turtlebot/TurtlebotStatus

In [ ]:
%%bash
source ~/robot_ws/devel/setup.bash
rossrv show TurtlebotStatus

[`rossrv`](http://wiki.ros.org/action/show/rosmsg?action=show&redirect=rossrv) is a tool for getting info about ROS Service types - [`rosmsg`](http://wiki.ros.org/action/show/rosmsg?action=show&redirect=rossrv#rosmsg-1) is the equivalent for ROS messages. After this, your ROS service is now ready to be used!

So let's use your `wander.py`! Make a copy and rename it to `wander_service.py`. You now need to declare and add the service callback and declaration. Our service will be advertised on the `turtlebot_status` service topic. For simplicity, you should modify your `ReadScan` class to contain the following new function:

``` python
def service_callback(self, rqst):
    rospy.loginfo("Requesting to change status? "+str(rqst.forwardrqst))
    self.driving_forward = rqst.forwardrqst
    ans = TurtlebotStatusResponse()
    ans.range = self.range_value
    ans.forwardresp = self.driving_forward
    return ans
```

Please note `driving_forward` is a class variable, so you need to update the rest of the script to reflect this (Note: Do remember to initialise `driving_forward` inside your class), and after the Subscriber and Publisher declarations add your service declaration as follows:

``` python
status_srv = rospy.Service('turtlebot_status', QueryRange, range_sensor.service_callback)
```

After you have finished writing your `wander_service.py` ROS node, let's fire-up a Turtlebot simulation if one isn’t already running:

``` bash
roslaunch turtlebot_gazebo turtlebot_world.launch
```

and then, in another console type:

``` bash
chmod +x ~/robot_ws/src/myturtlebot/src/wander_service.py
rosrun myturtlebot wander.py cmd_vel:=cmd_vel_mux/input/teleop
```

To test your new service, open a new console and type:

``` bash
rosservice call /turtlebot_status "forwardrqst: false"
```

As you can see, you can modify Turtlebot's behaviours and retrieve the range value when the behaviour's change happened. Again, if you get bored, you can change the parameters, play around with the script, so it does something more intelligent, or press Ctrl+C to stop it.

# Notes
_**<a id="note1"></a>NOTE 1:** to access the terminal within the docker container, open the tab containing the jupyter dashboard and select "new" then "terminal" - see Figure 5_

![jupyter-tree.png](imgs/jupyter-tree.png)
<div style="text-align:center"><b>Figure 5: How to open a terminal from Jupyter</b><br></div>

_**<a id="note2"></a>NOTE 2:** After opening a terminal within jupyter, you get only `$`, you need to enable bash, so type `bash` in the terminal and you should get something similar to Figure 6. Plese be aware that running a Linux container in Windows is really buggy, you will have a few crashes. To fix them, just restart the Docker daemon and start and attach to the container you currently have on your machine._

![bash_terminal.PNG](imgs/bash_terminal.PNG)
<div style="text-align:center"><b>Figure 6: Terminal within Jupyter</b><br></div>

_**<a id="note3"></a>NOTE 3 - Terminal Multiplexer:** After opening a terminal within jupyter, you can open multiple terminal tabs; however, this will become impractical at some point. For this, it is recommended to use TMUX, a terminal multiplexer that allows you to open panes within a terminal. First, type `tmux` and then try to open a pane by pressing `Ctrl+B "`. You can navigate between panes by pressing `Ctrl+B up arrow` or `Ctrl+B down arrow`. For more tricks, see https://gist.github.com/andreyvit/2921703_

## References
* Quigley, M., Gerkey, B. and Smart, W.D., 2015. Programming Robots with ROS: a practical introduction to the Robot Operating System. O'Reilly Media, Inc.